In [3]:
import os
print(os.getcwd())

/home/erica_2291/pb_proj


In [1]:
# 0: 데이터 7:2:1로 나누기 (훈련 train / 검증 val / test 테스트)

import os, shutil, random

def split_data_by_ratio(source_dir, class_name, output_root="UCF_Crime_Split",
                        split_ratio=(0.7, 0.2, 0.1)):
    files = [f for f in os.listdir(source_dir) if f.lower().endswith(('.mp4'))]
    random.shuffle(files)

    n_total = len(files)
    n_train = int(n_total * split_ratio[0])
    n_val = int(n_total * split_ratio[1])
    n_test = n_total - n_train - n_val

    splits = ['train', 'val', 'test']
    counts = [n_train, n_val, n_test]
    start = 0

    for split, count in zip(splits, counts):
        target_dir = os.path.join(output_root, split, class_name)
        os.makedirs(target_dir, exist_ok=True)
        for file in files[start:start+count]:
            shutil.copy2(os.path.join(source_dir, file), os.path.join(target_dir, file))
        start += count


In [4]:
split_data_by_ratio(
    source_dir="../2_unzipped/Anomaly-Videos-Part-2/Fighting",
    class_name="fighting",
    output_root="UCF_Crime_Split"
)

split_data_by_ratio(
    source_dir="../4_unzipped/Anomaly-Videos-Part-4/Vandalism",
    class_name="vandalism",
    output_root="UCF_Crime_Split"
)

split_data_by_ratio(
    source_dir="../normal/Testing_Normal_Videos_Anomaly",
    class_name="normal",
    output_root="UCF_Crime_Split"
)

In [ ]:
# 1: mediapipe로 포즈 학습

import cv2
import mediapipe as mp
import numpy as np
from tqdm import tqdm

def extract_pose_features(video_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False)
    cap = cv2.VideoCapture(video_path)

    features = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmark_vector = []
            for lm in results.pose_landmarks.landmark:
                landmark_vector.extend([lm.x, lm.y, lm.z, lm.visibility])
            features.append(landmark_vector)
        else:
            features.append([0.0] * (33 * 4))  # placeholder if no detection

    cap.release()
    return np.array(features)

In [ ]:
# 2: sklearn으로 관절 위치 분류(SVM)

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# 예시: 클래스별 벡터 추출
X_fighting = extract_pose_features("fighting.mp4")
X_vandalism = extract_pose_features("vandalism.mp4")
y_fighting = ['fighting'] * len(X_fighting)
y_vandalism = ['vandalism'] * len(X_vandalism)

# 데이터 결합
X = np.vstack([X_fighting, X_vandalism])
y = y_fighting + y_vandalism

# 정규화 + 학습
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

print(classification_report(y_test, clf.predict(X_test)))

In [ ]:
# 3: 학습한 모델로 이상행동 감지 확인

X_new = extract_pose_features("demo.mp4")
X_new_scaled = scaler.transform(X_new)
y_pred = clf.predict(X_new_scaled)

from collections import Counter
print("🎯 영상 예측:", Counter(y_pred).most_common(1)[0][0])